# Skip-Gram Model

- From [v1] Lecture 44

## Model Architecture

![Skip_Gram_Model_2](images/Skip_Gram_Model_2.jpg)

## Neural Network Architecture - A Sample

![Skip_Gram_Neural Network Architecture__A_Sample](images/Skip_Gram_Neural_Network Architecture__A_Sample.jpg)